In [1]:
# import internal .py modules
import file_path_management as fpath
import public_library as plib

2023-08-10 15:29:06 GM01X680 metapub.config[320085] WARNING NCBI_API_KEY was not set.


In [2]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re
from lxml import etree

In [3]:
# input_path = fpath.poten_litera_filled
# output_path = fpath.poten_litera_ids_filled
# plib.clear_file(output_path)

# df = pd.read_csv(input_path, header=None, sep = ",")
# df.columns = ["DOI", "PMID", "PMCID", "Title", "full_text_link", "pdf_link"]

# for ind in range(len(df)):
#     if df["DOI"][ind] == df["DOI"][ind]:
#         doi = str(df["DOI"][ind]).lower()
#         # df.set_value('DOI', ind, doi)
#         df.loc[ind, "DOI"] = doi
#     print(ind)

# df.to_csv(output_path, header=False, index=False)

In [4]:
# get pmid from title
def title2pmid(title):
    title = str(title).strip()
    words = title.split(" ")
    # print(words)
    term = ""
    for i in range(len(words)-1):
        term = term + words[i] + "+"
    term = term + words[-1]
    # print(term)
    url = "https://pubmed.ncbi.nlm.nih.gov/?term=" + term
    # print(url)
    soup = plib.request_webpage(url)
    try:
        pmid = soup.find_all("section", {"class": "matching-citations search-results-list"})[0].find_all("span", {"class": "docsum-pmid"})[0].get_text()
    except:
        pmid = np.nan
    if pmid != pmid:
        try:
            pmid = soup.find_all("ul", {"id": "full-view-identifiers"})[0].find_all("span", {"class": "identifier pubmed"})[0].find_all("strong", {"class": "current-id"})[0].get_text()
        except:
            pmid = np.nan
    if pmid == pmid:
        pmid = str(pmid).strip()
    return pmid
# --------------------start of test code--------------------
# pmid = "21434138"
title = "On the Organisation of the Visual Cortical Projection to the Pulvinar in Macaca mulatta (Part 2 of 2)"
# https://pubmed.ncbi.nlm.nih.gov/?term=Thalamocortical+connections+of+the+parabelt+auditory+cortex+in+macaque+monkeys
# title = "Independence and merger of thalamocortical channels within macaque monkey primary visual cortex: anatomy of interlaminar projections"
# title = "… of GABAB antagonist [3H] CGP 62349 binding in the rhesus monkey thalamus and basal ganglia and the influence of lesions in the reticular thalamic nucleus"
pmid = title2pmid(title)
print(pmid)
# ---------------------end of test code---------------------

4196833
